In [1]:
from keras.models import load_model
from keras.models import clone_model
from numpy import average
from numpy import array

Using TensorFlow backend.


In [2]:
# load models from file
def load_all_models(n_start, n_end):
    all_models = list()
    for epoch in range(n_start, n_end):
        # define filename for this ensemble
        filename = 'model_' + str(epoch) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('> Loaded %s' % filename)
    return all_models

In [3]:
# create a model from the weights of multiple models
def model_weight_ensemble(members, weights):
    # determine how many layers need to be averaged
    n_layers = len(members[0].get_weights())
    # create an set of average model weights
    avg_model_weights = list()
    for layer in range(n_layers):
        # collect this layer from each model
        layer_weights = array([model.get_weights()[layer] for model in members])
        # weighted average of weights for this layer
        avg_layer_weights = average(layer_weights, axis=0, weights=weights)
        # store average layer weights
        avg_model_weights.append(avg_layer_weights)
    # create a new model with the same structure
    model = clone_model(members[0])
    # set the weights in the new
    model.set_weights(avg_model_weights)
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                 metrics=['accuracy'])
    return model

In [4]:
# load all models into memory
members = load_all_models(490, 500)
print('Loaded %d models' % len(members))

W0102 12:47:13.400590 11312 deprecation_wrapper.py:119] From C:\Users\Lenovo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0102 12:47:13.426356 11312 deprecation_wrapper.py:119] From C:\Users\Lenovo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0102 12:47:13.506999 11312 deprecation_wrapper.py:119] From C:\Users\Lenovo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:178: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0102 12:47:13.511100 11312 deprecation_wrapper.py:119] From C:\Users\Lenovo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:185: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0102 12:47:13.511100 11312 deprecation_wrapper.py:119] From C:

> Loaded model_490.h5
> Loaded model_491.h5
> Loaded model_492.h5
> Loaded model_493.h5
> Loaded model_494.h5
> Loaded model_495.h5
> Loaded model_496.h5
> Loaded model_497.h5
> Loaded model_498.h5
> Loaded model_499.h5
Loaded 10 models


In [5]:
# prepare an array of equal weights
n_models = len(members)
weights = [1/n_models for i in range(1, n_models+1)]

In [6]:
weights

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [7]:
# create a new model with the weighted average of all model weights
model = model_weight_ensemble(members, weights)

In [8]:
# summarize the created model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 25)                75        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 78        
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________
